# Assignment: 1NN против RandomForest


В этом задании будет использоваться датасет `digits` из `sklearn.datasets`. Оставьте _последние_ 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами `1NN` с `RandomForestClassifier` из `sklearn` на 1000 деревьях.

In [8]:
from sklearn import datasets, model_selection, metrics, ensemble
from sklearn.neighbors import DistanceMetric
import numpy as np

In [9]:
X, y = datasets.load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, shuffle=False)

In [10]:
print(X_test.shape[0]/(X_test.shape[0]+X_train.shape[0]))

0.25041736227045075


## Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка $N log N$ сравнений (более строго говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. 

За выбор метода поиска ближайших соседей в `KNeigborsClassifier` из `sklearn` отвечает параметр `algorithm` — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных `ball tree` и `kd tree`.

Доля ошибок, допускаемых 1NN на тестовой выборке, — **ответ в задании 1**.

In [11]:
# считает растояние между точками
dist = DistanceMetric.get_metric('euclidean') 
nn_predict = []
for i in range(len(X_test)):
    #возвращает список расстояний от объекта X_test[0] до каждого объекта в выборке
    distances = dist.pairwise(np.vstack((X_test[i],X_train)))[0].tolist()
    #добавляет к списку расстояний колонку с метками классов
    indexed_dist = np.column_stack((distances[1:],y_train.tolist())).tolist()
    #сортирует список расстояний и возвращает метку класса ближайшего соседа
    nn_predict.append(sorted(indexed_dist)[0][1])

In [18]:
print("1NN acuracy = %.4f" % metrics.accuracy_score(y_test,nn_predict))
print("1NN error rate = %.4f" % (1 - metrics.accuracy_score(y_test,nn_predict)))
with open('ans1.txt', 'w') as f_out:
        f_out.write(str(1 - metrics.accuracy_score(y_test,nn_predict)))

1NN acuracy = 0.9622
1NN error rate = 0.0378


## Задание 2

Теперь обучите на обучающей выборке `RandomForestClassifier(n_estimators=1000)` из `sklearn`. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — **ответ в задании 2**. 

Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — `1NN`. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [13]:
rf = ensemble.RandomForestClassifier(n_estimators=1000)
rf.fit(X_train,y_train)
rf_predict = rf.predict(X_test)

In [19]:
print("RF accuracy = %.4f" % metrics.accuracy_score(y_test,rf_predict))
print("RF error rate = %.4f" % (1 - metrics.accuracy_score(y_test,rf_predict)))
with open('ans2.txt', 'w') as f_out:
        f_out.write(str(1 - metrics.accuracy_score(y_test,rf_predict)))

RF accuracy = 0.9267
RF error rate = 0.0733
